# Gradient-based optimization

In [ ]:
import drjit as dr
import mitsuba as mi


mi.set_variant('cuda_ad_rgb')


In [ ]:
dr.__file__

In [ ]:
scene_Adam = mi.load_file('scenes/living_room.xml')
scene_AdamV = mi.load_file('scenes/living_room.xml')
scene_AdamVC = mi.load_file('scenes/living_room.xml')
scene_veach = False
# key = 'WallsBSDF.brdf_0.reflectance.value'
key = 'PaintingBSDF.brdf_0.reflectance.data'




### Reference image

In [ ]:
image_ref = mi.render(scene_Adam)
# print(image_ref.numpy())

# Preview the reference image
mi.util.convert_to_bitmap(image_ref)
# mi.util.write_bitmap('/Users/yannizhang/Desktop/veach_ref.exr', image_ref)


In [ ]:
image_ref = mi.render(scene_Adam, spp=1, seed=1)
# print(image_ref.numpy())

# Preview the reference image
mi.util.convert_to_bitmap(image_ref)
# mi.util.write_bitmap('/Users/yannizhang/Desktop/veach_ref.exr', image_ref)

In [ ]:
params_Adam = mi.traverse(scene_Adam)
params_AdamV = mi.traverse(scene_AdamV)
params_AdamVC = mi.traverse(scene_AdamVC)
params_Adam

In [ ]:
new_p = mi.Bitmap('scenes/living-room/new_carrot.png')
bmp = new_p.convert(
    pixel_format=mi.Bitmap.PixelFormat.RGB,
    component_format=mi.Struct.Type.Float32,
    srgb_gamma=True
)

# Thanks to the array interface protocol, it is easy to create a TensorXf from a bitmap object
img = mi.TensorXf(bmp)

mi.util.convert_to_bitmap(img)

### Initial state

In [ ]:
# Save the original value
# param_ref_Adam = mi.Color3f(params_Adam[key])
# param_ref_AdamV = mi.Color3f(params_AdamV[key])
# param_ref_AdamVC = mi.Color3f(params_AdamVC[key])

param_ref_Adam = mi.TensorXf(params_Adam[key])
param_ref_AdamV = mi.TensorXf(params_AdamV[key])
param_ref_AdamVC = mi.TensorXf(params_AdamVC[key])

# Set another color value and update the scene
# params_Adam[key] = mi.Color3f(0.01, 0.2, 0.9)
# params_AdamV[key] = mi.Color3f(0.01, 0.2, 0.9)
# params_AdamVC[key] = mi.Color3f(0.01, 0.2, 0.9)

params_Adam[key] = img
params_AdamV[key] = img
params_AdamVC[key] = img


params_Adam.update();
params_AdamV.update();
params_AdamVC.update();

As expected, when rendering the scene again, the wall has changed color.

In [ ]:
image_init = mi.render(scene_Adam)
mi.util.convert_to_bitmap(image_init)

## Optimization

For gradient-based optimization, Mitsuba ships with standard optimizers including *Stochastic Gradient Descent* ([<code>SGD</code>][1]) with and without momentum, as well as [<code>Adam</code>][2] <cite data-cite="kingma2014adam">[KB14]</cite>. We will instantiate the latter and optimize our scene parameter with a learning rate of `0.05`. 

We then set the color to optimize on the optimizer, which will now hold a copy of this parameter and enable gradient tracking on it. During the optimization process, the optimizer will always perfom gradient steps on those variables. To propagate those changes to the scene, we need to call the `update()` method which will copy the values back into the `params` data structure. As always this method also notifies all objects in the scene whose parameters have changed, in case they need to update their internal state.

This first call to `params.update()` ensures that gradient tracking with respect to our wall color parameter is propagated to the scene internal state. For more detailed explanation on how-to-use the optimizer classes, please refer to the dedicated [how-to-guide][3].

[1]: https://mitsuba.readthedocs.io/en/latest/src/api_reference.html#mitsuba.ad.SGD
[2]: https://mitsuba.readthedocs.io/en/latest/src/api_reference.html#mitsuba.ad.Adam
[3]: https://mitsuba.readthedocs.io/en/latest/src/how_to_guides/use_optimizers.html

In [ ]:
opt_Adam = mi.ad.Adam(lr=0.05)
opt_AdamV = mi.ad.AdamV(lr=0.05)
opt_AdamVC = mi.ad.AdamVC(lr=0.05)


opt_Adam[key] = params_Adam[key]
opt_AdamV[key] = params_AdamV[key]
opt_AdamVC[key] = params_AdamVC[key]

params_Adam.update(opt_Adam);
params_AdamV.update(opt_AdamV);
params_AdamVC.update(opt_AdamVC);

At every iteration of the gradient descent, we will compute the derivatives of the scene parameters with respect to the objective function. In this simple experiment, we use the [*mean square error*][1], or $L_2$ error, between the current image and the reference created above.

[1]: https://en.wikipedia.org/wiki/Mean_squared_error

In [ ]:
def mse(image):
    return dr.mean(dr.sqr(image - image_ref))

In the following cell we define the hyper parameters controlling our optimization loop, such as the number of iterations:

In [ ]:
iteration_count = 200

In [ ]:
# IGNORE THIS: When running under pytest, adjust parameters to reduce computation time
import os
if 'PYTEST_CURRENT_TEST' in os.environ:
    iteration_count = 2

It is now time to actually perform the gradient-descent loop that executes 50 differentiable rendering iterations.

In [ ]:
def optimize(params, scene,opt,param_ref,iteration_count):
    errors = []
    err_init = dr.sum(dr.sqr(param_ref - params[key]))
    errors.append(err_init)
    for it in range(iteration_count):
        # Perform a (noisy) differentiable rendering of the scene
        image = mi.render(scene, params,spp=1, seed = 1)
        # Evaluate the objective function from the current rendered image
        loss = mse(image)

        # Backpropagate through the rendering process
        dr.backward(loss, flags = dr.ADFlag.BackPropVarianceCounter | dr.ADFlag.ClearVertices)

        # g = dr.graphviz_ad()
        # g.view()


        # Optimizer: take a gradient descent step
        opt.step()

        # Post-process the optimized parameters to ensure legal color values.
        opt[key] = dr.clamp(opt[key], 0.0, 1.0)

        # Update the scene state to the new optimized values
        params.update(opt)
        
        # Track the difference between the current color and the true value
        err_ref = dr.sum(dr.sqr(param_ref - params[key]))
        print(f"Iteration {it:02d}: parameter error = {err_ref[0]:6f}", end='\r')
        errors.append(err_ref)
    print('\nOptimization complete.')
    return errors

In [ ]:
errors_Adam = optimize(params_Adam, scene_Adam,opt_Adam,param_ref_Adam,iteration_count);
errors_AdamV = optimize(params_AdamV, scene_AdamV,opt_AdamV,param_ref_AdamV,iteration_count);
errors_AdamVC = optimize(params_AdamVC, scene_AdamVC,opt_AdamVC,param_ref_AdamVC,iteration_count);


## Results

We can now render the scene again to check whether the optimization process successfully recovered the color of the red wall.

In [ ]:
image_final = mi.render(scene_Adam, spp=128)
# mi.util.write_bitmap('/Users/yannizhang/Desktop/veavh_adam.exr', image_final)
mi.util.convert_to_bitmap(image_final)


In [ ]:
image_final = mi.render(scene_AdamV, spp=128)
# mi.util.write_bitmap('/Users/yannizhang/Desktop/veach_V.exr', image_final)
mi.util.convert_to_bitmap(image_final)

In [ ]:
image_final = mi.render(scene_AdamVC, spp=128)
# mi.util.write_bitmap('/Users/yannizhang/Desktop/veach_vc.exr', image_final)
mi.util.convert_to_bitmap(image_final)

It worked!

Note visualizing the objective value directly sometimes gives limited information, since differences between `image` and `image_ref` can be dominated by Monte Carlo noise that is not related to the parameter being optimized. 

Since we know the “true” target parameter in this scene, we can validate the convergence of the optimization by checking the difference to the true color at each iteration:

In [ ]:
import matplotlib.pyplot as plt

plt.plot(errors_Adam, linestyle='--',label='Adam')
plt.plot(errors_AdamV, linestyle='-',label='AdamV')
plt.plot(errors_AdamVC, linestyle=':',label='AdamVC')
plt.legend()

plt.xlabel('Iteration'); plt.ylabel('MSE(param)'); plt.title('Parameter error plot');
# plt.savefig('/Users/yannizhang/Desktop/veach_mis.png')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(errors_Adam[1:], linestyle='--',label='Adam')
plt.plot(errors_AdamV[1:], linestyle='-',label='AdamV')
plt.plot(errors_AdamVC[1:], linestyle=':',label='AdamVC')
plt.legend()

plt.xlabel('Iteration'); plt.ylabel('MSE(param)'); plt.title('Parameter error plot');
# plt.savefig('/Users/yannizhang/Desktop/RGL/5.10/scene2_adma_V_C.png')
plt.show()

In [ ]:
mi.util.convert_to_bitmap(params_Adam[key])


In [ ]:
mi.util.convert_to_bitmap(params_AdamV[key])


In [ ]:
mi.util.convert_to_bitmap(params_AdamVC[key])


## See also

- [Detailed look at <code>Optimizer</code>](https://mitsuba.readthedocs.io/en/latest/src/how_to_guides/use_optimizers.html)
- API reference:
  - [<code>mitsuba.ad.Optimizer</code>](https://mitsuba.readthedocs.io/en/latest/src/api_reference.html#mitsuba.ad.Optimizer)
  - [<code>mitsuba.ad.integrators.prb</code>](https://mitsuba.readthedocs.io/en/latest/src/api_reference.html#mitsuba.ad.integrators.prb)
  - [<code>mitsuba.ad.SGD</code>](https://mitsuba.readthedocs.io/en/latest/src/api_reference.html#mitsuba.ad.SGD)
  - [<code>mitsuba.ad.Adam</code>](https://mitsuba.readthedocs.io/en/latest/src/api_reference.html#mitsuba.ad.Adam)
  - [<code>drjit.backward</code>](https://drjit.readthedocs.io/en/latest/src/api_reference.html#drjit.backward)